# Evaluate original models and finetuned summarizers with ROUGE and BertScore

Fine-tuned and saved models on HF:

short-form models:
- Chung-Fan/bart-pubmed-20k (original: facebook/bart-large-cnn)

- Chung-Fan/distilbart-pubmed-20k (original: philschmid/distilbart-cnn-12-6-samsum)

- Chung-Fan/pegasus-pubmed-20k (original: tuner007/pegasus_summarizer)

long-form models:

- Chung-Fan/primera-pubmed-20k (original: allenai/PRIMERA)

- Chung-Fan/led-pubmed-20k (original: pszemraj/led-base-book-summary)

- Chung-Fan/longformer-pubmed-20k (original: hyesunyun/update-summarization-bart-large-longformer)

finetuned on bottom-truncated dataset:

- Chung-Fan/bart-pubmed-20k-bottom-tokens

- Chung-Fan/distilbart-pubmed-20k-bottom-tokens

- Chung-Fan/pegasus-pubmed-20k-bottom-tokens

---
** All the datasets, generated summaries, and evaluation results can be found in my shared Google Drive folder: https://drive.google.com/drive/folders/1sNoJxaShjifrt_AqyG5_sZYGxHknqfOM?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import transformers
import datasets
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

scaledown = 20000
summary_max_length = 400
my_num_beams = 3 # 8 to 3, to make it less computational intensive

# Replace model_index, my_model, and og_model with the other models listed in the intro text above to evaluate every model
model_index = "distilbart-pubmed-20k"
my_model = "Chung-Fan/distilbart-pubmed-20k"
og_model = "philschmid/distilbart-cnn-12-6-samsum"

notebook_login()

In [ ]:
!pip install datasets py7zr rouge_score bert_score

In [ ]:
from datasets import load_metric
from tqdm import tqdm

rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
bertscore_metric = load_metric("bertscore")
bertscore_names = ["precision", "recall", "f1"]

metrics = [rouge_metric, bertscore_metric]
metric_names = [str(rouge_metric.name), str(bertscore_metric.name)]

<ipython-input-4-b0c7f0d427bc>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


# Load processed dataset

In [ ]:
# Load the CSV file into a DataFrame
token_df = pd.read_csv('/content/drive/My Drive/pubmed/token_df.csv')
token_df_test = pd.read_csv('/content/drive/My Drive/pubmed/token_df_test.csv')
token_df_val = pd.read_csv('/content/drive/My Drive/pubmed/token_df_val.csv')

token1024_df = pd.read_csv('/content/drive/My Drive/pubmed/token1024_df.csv')
token1024_df_test = pd.read_csv('/content/drive/My Drive/pubmed/token1024_df_test.csv')
token1024_df_val = pd.read_csv('/content/drive/My Drive/pubmed/token1024_df_val.csv')

In [ ]:
# Convert DataFrames to Hugging Face Datasets
dataset_train = Dataset.from_pandas(token_df)
dataset_test = Dataset.from_pandas(token_df_test)
dataset_val = Dataset.from_pandas(token_df_val)

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
    'validation': dataset_val
})

dataset_med = dataset_dict
dataset_med

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article_len', 'abstract_len', 'article', 'abstract'],
        num_rows: 10700
    })
    test: Dataset({
        features: ['Unnamed: 0', 'article_len', 'abstract_len', 'article', 'abstract'],
        num_rows: 1125
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'article', 'abstract'],
        num_rows: 1107
    })
})

In [ ]:
# Convert DataFrames to Hugging Face Datasets
dataset1024_train = Dataset.from_pandas(token1024_df)
dataset1024_test = Dataset.from_pandas(token1024_df_test)
dataset1024_val = Dataset.from_pandas(token1024_df_val)

# Create DatasetDict
dataset1024_dict = DatasetDict({
    'train': dataset1024_train,
    'test': dataset1024_test,
    'validation': dataset1024_val
})

dataset1024_med = dataset1024_dict
dataset1024_med

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article', 'abstract'],
        num_rows: 642
    })
    test: Dataset({
        features: ['Unnamed: 0', 'article', 'abstract'],
        num_rows: 74
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'article', 'abstract'],
        num_rows: 56
    })
})

# Evaluate fine-tuned model

In [ ]:
!nvidia-smi

Fri Apr 12 11:27:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(my_model)
model = AutoModelForSeq2SeqLM.from_pretrained(my_model).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

In [ ]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def evaluate_summaries(dataset, metrics, model, tokenizer, generated_summaries,
                       batch_size=16, device=device,
                                   column_text="article",
                                   column_summary="highlights"):
    '''Calculate respective rouge metric for the given data'''
    generated_summaries = []

    article_batches = list(chunks(dataset[column_text], batch_size)) # dialogue batches
    target_batches = list(chunks(dataset[column_summary], batch_size))  # target batches

    i = 0
    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt") # encode the input
            print(type(article_batch))
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),  # generate summary
                             attention_mask=inputs["attention_mask"].to(device),
                             length_penalty=0.8, num_beams=my_num_beams, max_length=summary_max_length)
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True) for s in summaries] # decode them

            decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]  # misc processing

            for metric in metrics:
                metric.add_batch(predictions=decoded_summaries, references=target_batch)  # add this batch to the metric

    scores = dict()
    for metric in metrics:
      if str(metric.name) == 'bert_score':
        scores['bert_score'] = metric.compute(lang="en") # Calculate final metric score
      else:
        scores[str(metric.name)] = metric.compute() # Calculate final metric score

    return scores

In [ ]:
generated_summaries = []
scores_fineTuned = evaluate_summaries(dataset_med["test"], metrics , model, tokenizer, generated_summaries, column_text="article", column_summary="abstract", batch_size=8)
rouge_dict_fineTuned = dict((rn, scores_fineTuned['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_fineTuned = dict((rn, sum(scores_fineTuned['bert_score'][rn])/len(scores_fineTuned['bert_score'][rn])) for rn in bertscore_names)


  0%|          | 0/141 [00:00<?, ?it/s]

<class 'list'>


  1%|          | 1/141 [00:07<18:36,  7.98s/it]

<class 'list'>


  1%|▏         | 2/141 [00:16<18:51,  8.14s/it]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])

,rouge1,rouge2,rougeL,rougeLsum
distilbart-pubmed-20k,0.39833,0.162007,0.237544,0.348125


In [ ]:
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])

,f1,precision,recall
distilbart-pubmed-20k,0.852918,0.850995,0.855384


# Evaluate original model

In [ ]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model.

In [ ]:
model_ckpt = og_model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
scores_original = evaluate_summaries(dataset_med["test"], metrics, model, tokenizer, column_text="article", column_summary="abstract", batch_size=8)
rouge_dict_original = dict((rn, scores_original['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_original = dict((rn, sum(scores_original['bert_score'][rn])/len(scores_original['bert_score'][rn])) for rn in bertscore_names)

  0%|          | 0/141 [00:00<?, ?it/s]

<class 'list'>


  1%|          | 1/141 [00:03<08:27,  3.63s/it]

<class 'list'>


  1%|▏         | 2/141 [00:06<07:00,  3.02s/it]

...

 99%|█████████▊| 139/141 [04:50<00:04,  2.05s/it]

<class 'list'>


 99%|█████████▉| 140/141 [04:53<00:02,  2.18s/it]

<class 'list'>


100%|██████████| 141/141 [04:55<00:00,  2.09s/it]


In [ ]:
pd.DataFrame.from_records(rouge_dict_original, index=[f"model without finetuning"])


,rouge1,rouge2,rougeL,rougeLsum
model without finetuning,0.270585,0.097951,0.17736,0.227989


In [ ]:
pd.DataFrame.from_records(bertscore_dict_original, index=[f"model without finetuning"])

,f1,precision,recall
model without finetuning,0.831853,0.859441,0.806432


# Shorter text performance Evaluation
Now we want to evaluate the model's performance on shorter text (1024)

In [ ]:
scores_fineTuned_1024 = evaluate_summaries(dataset1024_med["test"], metrics , model, tokenizer, column_text="article", column_summary="abstract", batch_size=8)
rouge_dict_fineTuned_1024 = dict((rn, scores_fineTuned_1024['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_fineTuned_1024 = dict((rn, sum(scores_fineTuned_1024['bert_score'][rn])/len(scores_fineTuned_1024['bert_score'][rn])) for rn in bertscore_names)

pd.DataFrame.from_records(rouge_dict_fineTuned_1024, index=[model_index])

  0%|          | 0/10 [00:00<?, ?it/s]

<class 'list'>


 10%|█         | 1/10 [00:02<00:18,  2.02s/it]

<class 'list'>


 20%|██        | 2/10 [00:03<00:14,  1.85s/it]

...

<class 'list'>


 90%|█████████ | 9/10 [00:18<00:02,  2.06s/it]

<class 'list'>


100%|██████████| 10/10 [00:19<00:00,  1.94s/it]


,rouge1,rouge2,rougeL,rougeLsum
distilbart-pubmed-20k,0.325677,0.133961,0.217255,0.271944


In [ ]:
pd.DataFrame.from_records(bertscore_dict_fineTuned_1024, index=[model_index])

,f1,precision,recall
distilbart-pubmed-20k,0.843664,0.862857,0.825671


In [ ]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model.

original pretrained model on shorter text (1024)

In [ ]:
scores_original_1024 = evaluate_summaries(dataset1024_med["test"], metrics, model, tokenizer, column_text="article", column_summary="abstract", batch_size=8)
rouge_dict_original_1024 = dict((rn, scores_original_1024['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_original_1024 = dict((rn, sum(scores_original_1024['bert_score'][rn])/len(scores_original_1024['bert_score'][rn])) for rn in bertscore_names)

pd.DataFrame.from_records(rouge_dict_original_1024, index=[f"model without finetuning (1024 tokens)"])

  0%|          | 0/10 [00:00<?, ?it/s]

<class 'list'>


 10%|█         | 1/10 [00:01<00:16,  1.84s/it]

<class 'list'>


 20%|██        | 2/10 [00:03<00:14,  1.76s/it]

...

<class 'list'>


 90%|█████████ | 9/10 [00:18<00:02,  2.04s/it]

<class 'list'>


100%|██████████| 10/10 [00:19<00:00,  1.92s/it]


,rouge1,rouge2,rougeL,rougeLsum
model without finetuning (1024 tokens),0.325677,0.133961,0.217255,0.271944


In [ ]:
pd.DataFrame.from_records(bertscore_dict_original_1024, index=[f"model without finetuning (1024 tokens)"])

,f1,precision,recall
model without finetuning (1024 tokens),0.843664,0.862857,0.825671


In [ ]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model.

# Examine abstracts generated

We want to see with our own eyes and examine the generated abstracts to make sure they are doing what we want them to do.

In [ ]:
from transformers import pipeline

pipe = pipeline("summarization", model=my_model)

In [ ]:
for i in range(10):
  custom_article = str(token_df_test['article'][i])
  # print(pipe(custom_article, **gen_kwargs)[0]["summary_text"])
  print("human summary:\n", (token_df_test['abstract'][i]))
  print("generated summary:\n", pipe(custom_article[:1200])[0]["summary_text"]) # custom_article[:1200]

  print(i, ' : ---------------------------')

human summary:
 research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . 
 previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . 
 this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . 
 group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . 
 results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the trail 